In [1]:
"""
    Notebook to test the integration of price forecasting in SHIPP

    The goal is to change the function 'get_forecast_issue' to use the LEAR model instead of the simple AR model implemented here.

""" 

import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
from lear_48hrs import get_forecast_issue_lear
from animated_plot import plot_interactive_forecasts
from epftoolbox.data import read_data
from epftoolbox.models import LEAR
import pandas as pd
import os


In [3]:

def get_pt_forecast_autoreg(signal, autoreg, std_error, n, init_index = 0, seed = None):
    '''
    Function to generate a forecast based on the sum of observation and an autoregressive error

    Input:
        signal [list]: vector of observation data
        autoreg [list]: vector of the AR model parameters. The length of the list equals the order of the AR model
        std_error [list]: vector of the forecast error standard deviation as a function of the lead-time
        n [int]: Number of time steps in the forecast
        init_index [int]: offset for the observation data
        seed [int]: seed number to fix the random number generator

    Output:
        forecast [list]: vector of forecasted values
    '''

    # White noise term generation
    rng = np.random.default_rng(seed) 
    epsilon = rng.normal(size=n+init_index, scale=1)
    epsilon = epsilon[init_index:]

    # Initialisation of the AR error term
    x = [0 for _ in range(n)]

    for t in range(n): # Loop over time steps
        x[t] = std_error[t] * epsilon[t]  # First term of the AR model
        for j in range(len(autoreg)): # Loop to add the other terms of the AR model
            if t >= (j+1):
                x[t] += autoreg[j] * x[t-(j+1)]
    
    # Sum of the error term and the observations
    forecast = [signal[init_index+t] + x[t] for t in range(n)]

    return forecast


def get_forecast_issue(data_price, n_for, init_index):
    '''
    Dummy function to issue a forecast

    Input: 
        data_price [list]: list of hourly electricity price in EUR/MWh
        n_for [int]: number of time steps in the issue
        init_index [int]: time index of the issue time 

    Output:
        forecast_issue [list]: vector of forecasted values. Should have the dimension (n_for)
    '''
    # Input for the autoregressive function AR(2)
    autoreg = [0.8, -0.19]  # Model parameters
    std_error = [5 for _ in range(n_for)] # Standard deviation of the white noise term
    std_error[0] = 0
    seed = 0 # Seed for the random number generator

    forecast_issue = get_pt_forecast_autoreg(data_price, autoreg, std_error, n_for, init_index = init_index, seed = seed)
    return forecast_issue

def get_forecast_all(data_price, nt, n_for, dt_issue):
    ''' 
    Dummy function to combine each forecast issue into one object
    Input:
        data_price [list]: list of hourly electricity price in EUR/MWh
        nt [int]: number of forecast issues 
        n_for [int]: number of time steps in each forecast issue
        dt_issue [int]: time step duration between each forecast issue, in hours
    
    Output:
        forecast_vec [list]: list of forecast issues. Should have the following dimensions: (nt, 1, n_for)
    
    '''
    
    forecast_vec = []
    for i in range(0, nt):
        forecast_issue = get_forecast_issue(data_price, n_for, init_index = i*dt_issue)

        forecast_vec.append([forecast_issue])


    return forecast_vec


In [ ]:
# -------------------------------
# Compact user-defined parameters
# -------------------------------
year = 2019
forecast_days = 2          
calibration_window = 365*2     # default 365-day calibration window     
rolling_window_days = 60     # default 60-day rolling window
country_code = 'NL'
api_key = '1206c24a-2fb6-4ec3-a47f-0e1be10dacd4'

path_datasets_folder = Path('..') / 'datasets'
path_recalibration_folder = Path('..') / 'experimental_files'
n_for = 48 # Maximum number of time steps in the leadtime
model = LEAR(calibration_window=calibration_window)

dt_issue = 24 # Time step for the forecast issues in hour - here the forecast is issued 24h
dt_for = 1 # Time step in the forecast in hour - here hourly forecast


In [5]:
"""
    Notebook to test the integration of price forecasting in SHIPP

    The goal is to change the function 'get_forecast_issue' to use the LEAR model instead of the simple AR model implemented here.

"""

# Run the forecast
forecast_mean, forecast_p10, forecast_p90, actual_price = get_forecast_issue_lear(
    year,
    forecast_days,
    model,
    calibration_window,
    path_recalibration_folder,
    path_datasets_folder,
    rolling_window_days,
    api_key,
    country_code,
    forecast_horizon = n_for
)


Test datasets: 2018-11-02 00:00:00 - 2019-01-02 23:00:00
Skipping 2019-01-02 00:00:00 due to: Found array with 0 sample(s) (shape=(0, 247)) while a minimum of 1 is required by Lasso.


In [8]:
# -----------------------------
# Check the format and type of 'forecast_mean'
# -----------------------------
assert len(forecast_mean) == forecast_days
assert type(forecast_mean) == list

for forecast_issue in forecast_mean:
    assert len(forecast_issue) == 1
    assert type(forecast_issue) == list

    assert len(forecast_issue[0]) == n_for
    assert type(forecast_issue[0]) == list

# -----------------------------
# Check the format and type of 'forecast_p10'
# -----------------------------
assert len(forecast_p10) == forecast_days
assert type(forecast_p10) == list

for forecast_issue in forecast_p10:
    assert len(forecast_issue) == 1
    assert type(forecast_issue) == list

    assert len(forecast_issue[0]) == n_for
    assert type(forecast_issue[0]) == list

# -----------------------------
# Check the format and type of 'forecast_p90'
# -----------------------------
assert len(forecast_p90) == forecast_days
assert type(forecast_p90) == list

for forecast_issue in forecast_p90:
    assert len(forecast_issue) == 1
    assert type(forecast_issue) == list

    assert len(forecast_issue[0]) == n_for
    assert type(forecast_issue[0]) == list

# -----------------------------
# Check the format and type of 'actual_price'       
# -----------------------------
assert len(actual_price) == forecast_days
assert type(actual_price) == list

for forecast_issue in actual_price:
    assert len(forecast_issue) == 1
    assert type(forecast_issue) == list

    assert len(forecast_issue[0]) == n_for
    assert type(forecast_issue[0]) == list


In [5]:
plot_interactive_forecasts(forecast_mean, forecast_p10, forecast_p90, actual_price, dt_issue=24, n_for=48)

